# Pathways simulation examples

## A trivial example

First, we create a [YAML](https://www.codeproject.com/Articles/1214409/Learn-YAML-in-five-minutes) configuration file
with minimal (and unrealistic) parameters to run the simulation (the name of the file is `simple.yml`):

In [1]:
%%writefile simple.yml
# In the configuration of the shipement generator, we only use couple of ports, flowers, and ports
# because they are currently not used by the simulation anyway.
shipment:
  origins:
  - Netherlands
  - Mexico
  flowers:
  - Hyacinthus
  - Rosa
  # We set very low number of boxes and stems.
  boxes:
    min: 2
    max: 5
stems_per_box:
  default: 10
ports:
  - NY JFK CBP
  - FL Miami Air CBP
pest:
  # We use one of the simplest pest placement options.
  # Fixed infestation rate for each shipment 
  infestation_rate:
    distribution: fixed_value
    value: 0.05
  # Pest present in a portion of random boxes
  arrangement: random_box
  random_box:
    # Pests present in every second shipment on average
    probability: 0.5
    # Roughly a third of boxes should be infested
    ratio: 0.33
inspection:
  # Decision about sampling will be made using stems as a inspection unit.
  unit: stems
  # We will inspect 2% of stems.
  sample_strategy: percentage
  percentage:
    proportion: 0.02
  # Once we pick a box, we will inspect all of its content.
  within_box_pct: 1
  # Boxes for inspection are selected randomly.
  selection_strategy: random

Writing simple.yml


Then we create run the simulation generating only couple shipments.
The low number of boxes per shipment (`boxes`) and stems per box (`stems_per_box`) in the configuration and the low number
of shipments in the simulation allows us to show the individual stems
within each shipment using a symbol of a flower (`✿`) indicating ones with pest using a symbol of a bug (`🐛`) instead of a flower.
Change the seed value to see different random placements of the pests within the shipment
(for example, using `--seed=5`).
The `arrangement` is set to `random_box`, so you will always see pest present on the whole box
which is marked by pipe symbols (`|`) in the output.
We are using `python -m pathways` to run the simulation through its command line interface:

In [2]:
!python -m pathways --num-shipments=5 --config-file=simple.yml --pretty --seed=4

━━ Shipment ━━ Boxes: 5 ━━ Stems: 50 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Shipment ━━ Boxes: 3 ━━ Stems: 30 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Shipment ━━ Boxes: 4 ━━ Stems: 40 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Shipment ━━ Boxes: 3 ━━ Stems: 30 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Shipment ━━ Boxes: 4 ━━ Stems: 40 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿


Simulation parameters:
-----------------------
infestation:
	 type: fixed_value
	 infestation rate: 0.05
	 pest arrangement: random_box
inspection:
	 unit: stems
	 proportion of box ins

## A more complex example

We start again by preparing the configuration file. In this example, we use more realistic size of the shipments
by setting `boxes` and `stems_per_box`. We also use non-constant infestation rate by using the beta distribution.

In [3]:
%%writefile complex.yml
shipment:
  origins:
  - Netherlands
  - Mexico
  flowers:
  - Hyacinthus
  - Rosa
  boxes:
    min: 1
    max: 3000
stems_per_box:
  default: 200
ports:
  - NY JFK CBP
  - FL Miami Air CBP
pest:
  infestation_rate:
    distribution: beta
    value: 0.05
    parameters:
    - 4
    - 60
  arrangement: clustered
  clustered:
    max_stems_per_cluster: 400
    distribution: random
    parameters:
    - 800
inspection:
  # Here we are using boxes as a sample unit.
  unit: boxes
  # Here we use and configure hypergeometric sampling.
  sample_strategy: hypergeometric
  hypergeometric:
    detection_level: 0.05
    confidence_level: 0.95
    min_boxes: 1
  # We can keep the configuration for sample_strategy: percentage here.
  # It is not used by the simulation and we can easily switch to it by changing sample_strategy
  # without actually touching the configurations under percentage: or hypergeometric:.
  percentage:
    proportion: 0.02
  within_box_pct: 1
  # We can change how sample units are selected for inspection by changing selection_strategy
  # to either random or tailgate.
  selection_strategy: random

Writing complex.yml


Now we simulate more shipments and we ask for multiple stochastic runs to increase robustness of the result
(but we still keep both values low to get result quickly for this example): 

In [4]:
!python -m pathways --num-simulations=4 --num-shipments=60 --config-file=complex.yml --seed=4



Simulation parameters:
-----------------------
infestation:
	 type: beta
	 infestation distribution parameters: [4, 60]
	 pest arrangement: clustered
	 cluster width: 800 stems
	 maximum infested stems per cluster: 400 stems
inspection:
	 unit: boxes
	 proportion of box inspected: 1
	 sample strategy: hypergeometric
	 detection level: 0.05
	 selection strategy: random


Simulation results:
-----------------------
Avg. % shipments slipped: 0.42%
Avg. infestation rate: 0.058
Avg. infestation rate of slipped shipments: 0.017
Avg. infestation rate of intercepted shipments: 0.058
Avg. number of boxes opened per shipment:
	 to completion: 56
	 to detection: 9
Avg. number of stems inspected per shipment:
	 to completion: 11128
	 to detection: 1735
Avg. % sample completed if sample ends at detection: 15.60%
Avg. % infested stems unreported if sample ends at detection: 87.78%


To learn more about the configuration and the simulation itself start in the [readme file](https://github.com/ncsu-landscape-dynamics/pathways-simulation#readme) of the simulation repository.